In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import HumanMessage
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings

import logging
import os

os.environ["DASHSCOPE_API_KEY"] = ''
logging.basicConfig(level=logging.INFO)

In [ ]:

embeddings = embeddings = OllamaEmbeddings(model="nomic-embed-text")
persist_directory = "./chroma_db"
chroma_db = Chroma(
    embedding_function=embeddings,
    persist_directory=persist_directory
)
logging.info('vectorstore created')

In [ ]:
# 添加文档到向量数据库
loader = PyPDFLoader("docs/DeepSeek-V2.pdf")
documents = loader.load()
logging.info(f"加载 {len(documents)} 个文档")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)
logging.info(f"文档共分割为 {len(split_docs)} 块")
chroma_db.add_documents(documents=split_docs)

In [ ]:
llm = ChatTongyi(streaming=True, model='qwen3-1.7b')
res = llm.invoke("解释一下DeepSeek-MOE架构", streaming=True)
print(res.content)

In [ ]:
# 示例查询


rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",            # 直接合并上下文
    retriever=chroma_db.as_retriever(),  # 使用ChromaDB检索器
    return_source_documents=True   # 返回源文档
)
# 执行RAG查询
result = rag_chain.invoke("解释一下DeepSeek-MOE架构")
print(result['result'])
print('\n--------------------------------------------------refered documents:--------------------------------------------------\n')
for doc in result['source_documents']:
    print(doc.page_content)
    print('--------------------------------------------------\n')